In [131]:
import pandas as pd
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
print("Setup Complete")

Setup Complete


# Context

In [270]:
pd.set_option('display.max_rows', 20)
sample = pd.read_csv('data/sample_submission.csv')
test = pd.read_csv('data/test.csv')
train = pd.read_csv('data/train.csv')
display(train)
# pd.set_option('display.max_rows', None)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,1,1,"Oconnor, Frankie",male,NaN,2,0,209245,27.14,C12239,S
1,1,0,3,"Bryan, Drew",male,NaN,0,0,27323,13.35,NaN,S
2,2,0,3,"Owens, Kenneth",male,0.33,1,2,CA 457703,71.29,NaN,S
3,3,0,3,"Kramer, James",male,19.00,0,0,A. 10866,13.04,NaN,S
4,4,1,3,"Bond, Michael",male,25.00,0,0,427635,7.76,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99995,1,2,"Bell, Adele",female,62.00,0,0,PC 15008,14.86,D17243,C
99996,99996,0,2,"Brown, Herman",male,66.00,0,0,13273,11.15,NaN,S
99997,99997,0,3,"Childress, Charles",male,37.00,0,0,NaN,9.95,NaN,S
99998,99998,0,3,"Caughlin, Thomas",male,51.00,0,1,458654,30.92,NaN,S


# Data quality assessment

In [274]:
assert all(train['PassengerId'] == np.arange(10 ** 5)), 'PassengerId must be in the range from 0 to 100000 inclusive'
assert all(train['Survived'].apply(lambda survived: survived in [0, 1])), 'Survived must have a value equal to 0 or 1'
assert all(train['Pclass'].apply(lambda pclass: pclass in range(1, 4))), \
'Pclass must have a value in the range from 1 to 3 inclusive'
assert all(train['Name'].str.contains('[A-Z]{1}[a-z]*, [A-Z]{1}[a-z]*', regex=True)), 'The Name must match the template'
assert all(train['Sex'].apply(lambda gender: gender in ['male', 'female'])), 'The Sex must be equal to either male or female'
assert all(train['Age'].astype(str).str.contains('\d\d?.\d\d?|nan', regex=True)), \
'Age must match the presented template and is equal to NaN'
assert all(train['SibSp'].apply(lambda sibsp: sibsp in range(9))), 'SubSp is not in the acceptable range'
assert all(train['Parch'].apply(lambda parch: parch in range(10))), 'Parch is not in the acceptable range'
assert all(train['Ticket'].astype(str).str.contains(
    '\d{4,5}|[A-Z].[A-Z5].?|[A-Z4]{2}.?|A/[4,5]|SOTON/O.Q.|W./C.|SC/PARIS|C|nan', regex=True)), \
'''Ticket must match one of the templates:
4 digits, 5 digits, letter.letter, 2 letters, A.5., A4., A/4, A/5, SOTON/O.Q., W./C., SC/PARIS, C'''
assert all(train['Cabin'].astype(str).str.contains('[A-GT]\d{4,5}|nan', regex=True)), \
'Cabin must match one of the templates: letter from A to G or T and 4 or 5 digits'
assert all(train['Embarked'].astype(str).apply(lambda embarked: embarked in ['C', 'Q', 'S', 'nan'])), \
'Embarked must be equal to C, Q, S or NaN'
train = train.astype({'PassengerId': 'int64',
                      'Survived': 'int64',
                      'Pclass': 'int64',
                      'Name': 'str',
                      'Sex': 'str',
                      'Age': 'float64',
                      'SibSp': 'int64',
                      'Parch': 'int64',
                      'Ticket': 'str',
                      'Fare': 'float64',
                      'Cabin': 'str',
                      'Embarked': 'str'})
que = train.query('Cabin != "nan"').sort_values(by='Cabin').duplicated(subset='Cabin', keep=False)
que.name = 'que'
train_test = train.merge(que, left_index=True, right_index=True).query('que == True').srot
display(train_test)
print('The testing was completed successfully')


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,que
12,12,0,3,"Stewart, Pat",male,36.0,0,0,434108,9.34,C8674,S,True
14,14,1,1,"Seaton, Karen",female,44.0,0,1,12425,28.40,D6945,C,True
19,19,1,1,"Hall, Stephanie",female,61.0,0,4,PC 174821,205.01,B7010,Q,True
25,25,0,1,"Dagostino, Christa",female,53.0,1,1,11856,82.57,B11790,Q,True
27,27,1,1,"Way, Tina",female,35.0,0,2,476162,154.61,C20297,C,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99934,99934,1,1,"Benavides, Jack",male,51.0,0,0,320074,70.55,B9505,S,True
99937,99937,0,1,"Ussery, Ralph",male,60.0,0,0,PC 195006,24.13,B3794,Q,True
99942,99942,1,1,"Wilder, Sharon",female,30.0,0,0,463708,109.23,C13863,C,True
99949,99949,1,1,"Nolan, Nichole",female,49.0,1,1,S.O.C. 12919,148.73,B15425,S,True


The testing was completed successfully


# Data exploration

In [240]:
df = train['PassengerId'] == np.arange(10 ** 5)
display(df)

0        True
1        True
2        True
3        True
4        True
         ... 
99995    True
99996    True
99997    True
99998    True
99999    True
Name: PassengerId, Length: 100000, dtype: bool

# Summary